In [2]:
import pandas as pd
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options
from tqdm import tqdm
import logging
import re
import random
from urllib.parse import urlparse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl.utils import get_column_letter  
import win32api
import win32con

In [3]:
# -----------------------
# 1. 日志配置
# -----------------------
log_file = 'download_errors.log'
logging.basicConfig(
    filename=log_file,
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.ERROR
)

# -----------------------
# 2. 路径和文件名配置
# -----------------------
excel_file = r"D:\学习\Naraka\加勒比地区数据\总新闻资讯.xlsx" 
sheet_name = 'Sheet1'         # 需处理的工作表名称
output_dir = r"D:\学习\Naraka\加勒比地区数据\网页快照"
updated_excel_file = r"D:\学习\Naraka\加勒比地区数据\总新闻资讯网页快照.xlsx"
use_selenium = False           

# -----------------------
# 3. 列索引定义（从0开始）
# -----------------------
name_column_index = 1          
url_column_index = 9           
link_column_index = 10          

# -----------------------
# 4. 选择路径类型
# -----------------------
use_relative_path = True       

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# -----------------------
# 5. 读取 Excel 文件
# -----------------------
try:
    df = pd.read_excel(excel_file, sheet_name=sheet_name, header=0)  
except FileNotFoundError:
    logging.error(f"未找到 Excel 文件: {excel_file}")
    raise
except ValueError as ve:
    logging.error(f"读取 Excel 文件时出错: {ve}")
    raise

# 打印列名称以确认
print("Excel 文件中的列名称：", df.columns.tolist())

# -----------------------
# 6. 初始化 WebDriver
# -----------------------
if use_selenium:
    edge_options = Options()
    edge_options.use_chromium = True
    edge_options.add_argument('--headless')   # 无头模式
    edge_options.add_argument('--disable-gpu')
    edge_options.add_argument('--no-sandbox')
    edge_options.add_argument('--window-size=1920,1080')

    service = EdgeService(executable_path=r"D:\学习\msedgedriver.exe")  
    try:
        driver = webdriver.Edge(service=service, options=edge_options)
    except Exception as e:
        logging.error(f"初始化 Selenium WebDriver 失败: {e}")
        raise

# -----------------------
# 7. 辅助函数
# -----------------------
def sanitize_filename(name):
    """
    移除非法字符并限制文件名长度。
    """
    # 替换非法字符
    sanitized = re.sub(r'[<>:"/\\|?*]', '_', name)
    sanitized = sanitized.replace('【', '').replace('】', '')  # 删除特定字符
    sanitized = sanitized.replace('：', '')  # 删除冒号
    return sanitized[:255]  # 确保文件名不超过255个字符

def download_html_requests(url, name, row_index):
    """
    使用 requests 下载网页内容，带超时和错误处理。
    使用标题作为文件名，确保唯一性。
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/113.0.0.0 Safari/537.36'
        )
    }
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        html_content = response.text
        sanitized_name = sanitize_filename(name)
        # 通过添加行号确保文件名唯一
        file_name = f"{row_index-1}_{sanitized_name}.html"
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        return file_path
    except requests.exceptions.RequestException as e:
        logging.error(f"请求失败 {url}: {e}")
        return None
# 网页写入新文件并保存
def download_html_selenium(url, name, row_index, driver):
    """
    使用 Selenium 下载渲染后的网页内容并保存为 HTML 文件。
    使用标题作为文件名，确保唯一性。
    """
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        # 简单等待 body 标签出现
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))
        html_content = driver.page_source
        sanitized_name = sanitize_filename(name)
        # 通过添加行号确保文件名唯一
        file_name = f"{row_index-1}_{sanitized_name}.html"
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        return file_path
    except Exception as e:
        logging.error(f"Selenium 下载失败 {url}: {e}")
        return None
def login_to_cnki(driver, username, password):
    """
    使用 Selenium 模拟登录 CNKI 网站
    """
    try:
        # 打开 CNKI 登录页面
        driver.get("https://login.cnki.net/")

        # 填充CNKI用户名
        driver.find_element(By.ID, "××××@qq.com").send_keys(username)
        # 填写CNKI密码
        driver.find_element(By.ID, "×××××").send_keys(password)

        # 点击登录按钮
        driver.find_element(By.ID, "btnLogin").click()

        # 等待登录完成，确认页面加载
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "user-avatar")))  # 示例

        print("成功登录 CNKI")
    except Exception as e:
        logging.error(f"登录失败: {e}")
        print(f"登录失败，详情见日志文件。")
        raise


def load_cookies(driver, cookies):
    """
    加载并设置浏览器的 cookies
    """
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.refresh()  # 刷新页面，以应用 cookies

def insert_attachment(sheet, cell_address, file_path, attachment_name):
    """
    使用超链接插入附件到 Excel。
    """
    try:
        # 获取单元格的位置
        cell = sheet.range(cell_address)
        # 插入超链接
        cell.add_hyperlink(file_path, text_to_display=attachment_name)
        print(f"已在单元格 {cell_address} 插入本地文件超链接: {attachment_name}")
    except Exception as e:
        logging.error(f"在单元格 {cell_address} 插入本地文件超链接失败: {e}")
        print(f"在单元格 {cell_address} 插入本地文件超链接失败，详情见日志文件。")

def open_excel():
    import xlwings as xw  # 延迟导入 xlwings
    try:
        app = xw.App(visible=False)  # 设置为 True 可见
        wb_xlwings = app.books.open(excel_file)
        sheet_xlwings = wb_xlwings.sheets[sheet_name]
        return app, wb_xlwings, sheet_xlwings
    except Exception as e:
        logging.error(f"打开 Excel 文件失败: {e}")
        print("打开 Excel 文件失败。请检查日志文件以获取详细信息。")
        if use_selenium:
            driver.quit()
        raise
def fix_url(url):
    """确保 URL 以 http:// 或 https:// 开头"""
    if not url.startswith(("http://", "https://")):
        return "https:" + url  # 默认加 https
    return url
    
def process_excel():
    app, wb_xlwings, sheet_xlwings = open_excel()
    # CNKI 登录
    try:
        login_to_cnki(driver, "your_username", "your_password")
    except Exception as e:
        print("登录过程遇到问题，跳过登录")
    try:
        for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="下载进度"):
            # Excel 中的实际行号（假设第一行为标题）
            excel_row = index + 2  # openpyxl 和 xlwings 行号从1开始，且第一行为标题

            # 使用基于位置的索引访问名称和 URL
            try:
                name = row.iloc[name_column_index]   
                url = row.iloc[url_column_index] 
                url = fix_url(url)
            except IndexError as e:
                logging.error(f"列索引错误: {e} 在行 {excel_row}")
                print(f"列索引错误: {e} 在行 {excel_row}")
                continue

            # 验证 URL
            if pd.isna(url) or not isinstance(url, str) or not url.strip().startswith(('http://', 'https://')):
                print(f"第 {excel_row} 行的 URL 无效，跳过。")
                logging.error(f"第 {excel_row} 行的 URL 无效: {url}")
                continue

            # 验证名称
            if pd.isna(name) or not isinstance(name, str) or not name.strip():
                print(f"第 {excel_row} 行的名称无效，跳过。")
                logging.error(f"第 {excel_row} 行的名称无效: {name}")
                continue

            try:
                print(f"正在处理第 {excel_row} 行: 标题='{name}', URL='{url}'")

                # 下载网页内容
                if use_selenium:
                    print('YES___________________')
                    html_file_path = download_html_selenium(url, name, excel_row, driver)
                else:
                    html_file_path = download_html_requests(url, name, excel_row)

                if not html_file_path or not os.path.exists(html_file_path):
                    print(f"第 {excel_row} 行下载失败，文件不存在。")
                    logging.error(f"第 {excel_row} 行下载失败，文件路径：{html_file_path}")
                    continue
            # 插入附件到 Excel
                link_column_num = link_column_index + 1  # E列是5
                cell_address = f"{get_column_letter(link_column_num)}{excel_row}"
                attachment_name = f"{name}.html"
                insert_attachment(sheet_xlwings, cell_address, html_file_path, attachment_name)

                # 添加随机延迟，防止请求过于频繁
                time.sleep(random.uniform(1, 3))

            except Exception as e:
                logging.error(f"第 {excel_row} 行处理失败: {e}")
                print(f"第 {excel_row} 行处理失败，详情见日志文件。")
                continue

    finally:
        # -----------------------
        # 10. 关闭 WebDriver 和保存 Excel 文件
        # -----------------------
        if use_selenium:
            driver.quit()
        try:
            wb_xlwings.save(updated_excel_file)
            print(f"已保存更新后的 Excel 文件: {updated_excel_file}")
        except Exception as e:
            logging.error(f"保存更新后的 Excel 文件失败: {e}")
            print("保存更新后的 Excel 文件失败。请检查日志文件以获取详细信息。")
        finally:
            wb_xlwings.close()
            app.quit()

if __name__ == "__main__":
    process_excel()

Excel 文件中的列名称： ['Unnamed: 0', '资料标题', '事件', '地点', '主要描述内容', '事件起因', '事件结果', '事件影响', '来源', '网址', '网页快照']
登录过程遇到问题，跳过登录


下载进度:   0%|                                                                               | 0/2488 [00:00<?, ?it/s]

正在处理第 2 行: 标题='中国和古巴两国外交部签署档案领域合作协议', URL='https://www.mfa.gov.cn/web/wjbxw_673019/202412/t20241217_11495138.shtml'
已在单元格 K2 插入本地文件超链接: 中国和古巴两国外交部签署档案领域合作协议.html


下载进度:   0%|                                                                     | 1/2488 [00:01<1:08:56,  1.66s/it]

正在处理第 3 行: 标题='王沪宁会见古巴全国人民政权代表大会主席拉索', URL='https://www.mfa.gov.cn/web/zyxw/202412/t20241218_11497810.shtml'
已在单元格 K3 插入本地文件超链接: 王沪宁会见古巴全国人民政权代表大会主席拉索.html


...

正在处理第 2488 行: 标题='王毅将访问埃及、突尼斯、多哥、科特迪瓦、巴西、牙买加', URL='http://cn.chinadiplomacy.org.cn/2024-01/11/content_116934614.shtml'
已在单元格 K2488 插入本地文件超链接: 王毅将访问埃及、突尼斯、多哥、科特迪瓦、巴西、牙买加.html


下载进度: 100%|█████████████████████████████████████████████████████████████████▉| 2487/2488 [1:39:17<00:01,  1.98s/it]

正在处理第 2489 行: 标题='中国空军赴哈瓦那看望古巴老爷爷', URL='https://k.sina.com.cn/article_7517400647_1c0126e4705906mr72.html'
已在单元格 K2489 插入本地文件超链接: 中国空军赴哈瓦那看望古巴老爷爷.html


下载进度: 100%|██████████████████████████████████████████████████████████████████| 2488/2488 [1:39:20<00:00,  2.40s/it]


已保存更新后的 Excel 文件: D:\学习\Naraka\加勒比地区数据\总新闻资讯网页快照.xlsx
